In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_163.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_308.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_086.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_123.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_236.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_214.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_060.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_156.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_154.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_168.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_153.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_225.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_032.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_139.npy
/kaggle/input/mri-data/Data/Train/masks_reformatted/mask_120.npy
/kaggle/input/mri-data/Da

In [2]:
import tensorflow as tf
from keras.layers import Conv3D, ConvLSTM3D, Reshape, Conv3DTranspose, UpSampling3D, MaxPooling3D, Concatenate, Input
from keras.layers import Bidirectional, BatchNormalization, Dropout, Dense, Flatten, Attention, Add, Activation,ConvLSTM2D
from keras.models import Model
import keras
import matplotlib.pyplot as plt
from keras.metrics import categorical_crossentropy
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,Adadelta
from glob import glob


In [3]:
#Get corresponsing Mask id
def get_id(path):
    slice = path.split(".")[2]
    id = slice[len(slice)-1:len(slice)-4:-1]

    return id[-1::-1]

In [4]:
from sklearn.decomposition import PCA,IncrementalPCA
def pca_preprocessing(sample):
    #divide into its channels
    t1ce,t2,flair=sample[:,:,:,0],sample[:,:,:,1],sample[:,:,:,2]
    t1ce_f=np.reshape(t1ce,(128,128*128))
    t2_f=np.reshape(t2,(128,128*128))
    flair_f=np.reshape(flair,(128,128*128))
    
    p=PCA()
    #Get components that explains 99% of the slice
    #t1ce
    p.fit(t1ce_f)

    var1=np.cumsum(p.explained_variance_ratio_)*100
    k1=np.argmax(var1>99)
    
    #t2
    p.fit(t2_f)
    var2=np.cumsum(p.explained_variance_ratio_)*100
    k2=np.argmax(var2>99)
    
    #flair
    p.fit(flair_f)
    var3=np.cumsum(p.explained_variance_ratio_)*100
    k3=np.argmax(var3>99)
    
    #perform pca for k1,k2,k3 components
    pca_t1ce=PCA(k1)
    t1ce_t=pca_t1ce.inverse_transform(pca_t1ce.fit_transform(t1ce_f))
    
    pca_t2=PCA(k2)
    t2_t=pca_t2.inverse_transform(pca_t2.fit_transform(t2_f))
    
    pca_flair=PCA(k3)
    flair_t=pca_flair.inverse_transform(pca_flair.fit_transform(flair_f))
    
    #rebuild the image
    t1ceR=np.reshape(t1ce_t,(128,128,128))
    t2R=np.reshape(t2_t,(128,128,128))
    flairR=np.reshape(flair_t,(128,128,128))
    
    #Stack the channels
    sample_reduced=np.stack([t1ceR,t2R,flairR],axis=3)
    
    return sample_reduced
    
    

In [5]:
#Load all helper functions onto notebook
#Image generator
def generate_image(paths):
    for img_path in paths:
        example_id = get_id(img_path)
        sample = np.load(img_path)

        mask = np.load("../input/mri-data/Data/Train/masks_reformatted/mask_"+example_id+".npy","r")
        
        # Normalization
        normalized_sample = (sample-np.mean(sample))/np.std(sample)
        normalized_sample=np.round(normalized_sample,5)
        
        #flattern mask
        #mask=mask.flatten()
        yield normalized_sample, mask


In [6]:
#Batch generator

def batch_generator(paths, batch_size):
    while True:
        image = generate_image(paths)

        img_batch = list()
        mask_batch = list()

        for sample, mask in image:
            img_batch.append(sample)
            mask_batch.append(mask)

            if (len(img_batch) == batch_size):
                yield np.stack(img_batch, axis=0), np.stack(mask_batch, axis=0)
                img_batch = []
                mask_batch = []

        if (len(img_batch) != 0):
            yield np.stack(img_batch, axis=0), np.stack(mask_batch, axis=0)
            img_batch = []
            mask_batch = []

In [7]:
#Metrics 
from keras.losses import categorical_crossentropy
def dice_coef(y_true, y_pred):
    ytrue_f = keras.backend.flatten(y_true)
    ypred_f = keras.backend.flatten(y_pred)

    intersection = keras.backend.sum(
        ytrue_f*ypred_f)

    E = keras.backend.epsilon()
    dice = (2.*intersection+E)/(keras.backend.sum(ytrue_f)+keras.backend.sum(ypred_f)+E)
    return dice

def dice_loss_multilabel(y_true,y_pred):
    dice=0
    for i in range(4):
        dice=dice-dice_coef(y_true[:,:,:,i],y_pred[:,:,:,i])
    return dice

def dice_coef_multilabel(y_true,y_pred):
    dice=0
    for i in range(4):
        dice=dice+dice_coef(y_true[:,:,:,i],y_pred[:,:,:,i])
    return dice
    

def dice_loss(y_true, y_pred):
    dice_loss = 1-dice_coef(y_true, y_pred)
    return dice_loss


def jacard(y_true,y_pred):
    ytrue_f=keras.backend.flatten(y_true)
    ypred_f=keras.backend.flatten(y_pred)
    
    E=keras.backend.epsilon()
    
    intersection=keras.backend.abs(keras.backend.sum(ytrue_f*ypred_f))
    union=keras.backend.sum(keras.backend.abs(ytrue_f)+keras.backend.abs(ypred_f))-intersection
    
    jac=(intersection+E)/(union+E)
    
    return (1-jac)*E
    

def overall_loss(y_true, y_pred):
    loss = categorical_crossentropy(y_true, y_pred) + \
        dice_loss_multilabel(y_true, y_pred)
    return loss

def multilabel_loss(y_true,y_pred,m):
    dice=0
    for i in range(m):
        dice+=dice_coef(y_true[:,:,:,i],y_pred[:,:,:,i])
    return dice


In [112]:
#Metrics to handle class imbalance
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight


def get_class_weight(y_true):
    #convert one hot encoded y_true to categorical data
    t=y_true.shape
    res=tf.reshape(y_true,(128*128*128,4))
        
    cat=list()
    for i in y_true:
        c=0
        for k in i:
            if k==1:
                cat.append(c)
                break
            c+=1
        
    #compute the class weight
    print(cat)
    y,idx,count=tf.unique_with_counts(cat)
    
    print(y)
    print(count)
    f=dict(zip(y,count))
    
    #weights for each class
    total_samples=len(cat)
        
    w_bg=1/(total_samples/(4*f[0]))**2
    w_one=1/(total_samples/(4*f[1]))**2
    w_two=1/(total_samples/(4*f[2]))**2
    w_three=1/(total_samples/(4*f[3]))**2
        
    class_weights=[w_bg,w_one,w_two,w_three]
    return class_weights

def generalized_label_dice_loss(y_true_lab,y_pred_lab,cw):
        ytrue_f=keras.backend.flatten(y_true_lab)
        ypred_f=keras.backend.flatten(y_pred_lab)
        
        E=keras.backend.epsilon()
        intersection=cw*(keras.backend.sum(ytrue_f*ypred_f))
        union=cw*(keras.backend.sum(ytrue_f)+keras.backend.sum(ypred_f))
        
        dice_label=(intersection+E)/(union+E)
        
        return dice_label
    
def total_gen_dice_loss(ytrue,ypred):
    
    weights=get_class_weight(ytrue)
    
    s=0
    for i in range(4):
        s=s+generalized_label_dice_loss(ypred[:,:,:,:,i],ytrue[:,:,:,:,i],class_weights[i])
    
    loss=1-(2*s)
    return loss


def total_gen_dice_coef(ytrue,ypred):
        dice_coef=1-total_gen_dice_loss(ypred,ytrue)
        return dice_coef


In [110]:
k=np.array([1,2,2,1])
k[k==2]

array([2, 2])

In [80]:
a=tf.constant([1,2])

In [103]:
k=tf.where(a==1)

In [104]:
print(k[0][0])

tf.Tensor(0, shape=(), dtype=int64)


In [44]:
#Learning rate scheduler
def scheduler(epoch,lr):
    if(epoch%5==0):
        return lr*tf.math.exp(-0.1*epoch)
    else:
        return lr

In [45]:
#2 path CNN model
def create_2pathed_shallow():
    input_layer = Input((9,9,128, 3))
    # first encoding block
    # Convolution layers
    conv1_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv1_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_1)
    conv1_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_2)

    # Downsample
    pool1_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv1_e_3)

    # Convolution Layers
    conv1_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool1_e_1)
    conv1_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_4)
    conv1_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), strides=1, activation='elu', use_bias=True, bias_initializer='he_normal')(conv1_e_5)

    # Batch normalization to get mean 0 activations
    batch1 = BatchNormalization()(conv1_e_6)

    # Second encoding block
    # Convolutions
    conv2_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv2_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_1)
    conv2_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_2)

    # Pooling
    pool2_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv2_e_3)

    # Convolutions
    conv2_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool2_e_1)
    conv2_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_4)
    conv2_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_5)

    batch2 = BatchNormalization()(conv2_e_6)

    # Concatenate
    path_combined = Concatenate(axis=4)([batch1, batch2])

    # Combined convolutions
    conv_7 = Conv3D(filters=32, kernel_initializer='he_normal', kernel_size=(3, 3, 3), activation='elu',
                    strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(path_combined)
    conv_8 = Conv3D(filters=64, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_7)
    conv_9 = Conv3D(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_8)

    batch3 = BatchNormalization()(conv_9)
    
    #Birectional lstm layers
    

    # Deconvolution
    d_conv = Conv3DTranspose(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, use_bias=True, bias_initializer='he_normal')(batch3)
    d_conv_1 = Conv3DTranspose(filters=64, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv)
    d_conv_2 = Conv3DTranspose(filters=32, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_1)

    # first decoding block
    dconv1_e_1 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_2)
    dconv1_e_2 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_1)
    dconv1_e_3 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_2)

    upsample_e_1 = UpSampling3D(size=(2, 2, 2))(dconv1_e_3)

    dconv1_e_4 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(upsample_e_1)
    dconv1_e_5 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_4)

    batchnorm = BatchNormalization()(dconv1_e_5)

    mask = Conv3DTranspose(filters=4, kernel_initializer='glorot_normal', kernel_size=(
        3, 3, 3), activation='softmax', strides=1, padding='valid', use_bias=True, bias_initializer='glorot_normal')(batchnorm)

    model = Model(inputs=input_layer, outputs=mask)
    return model


In [46]:
#Bidirectional 2 path cnn-lstm model

def two_path_bi_lstm():
    input_layer = Input((128, 128, 128, 3))
    # first encoding block
    # Convolution layers
    conv1_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv1_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_1)
    conv1_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_2)

    # Downsample
    pool1_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv1_e_3)

    # Convolution Layers
    conv1_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool1_e_1)
    conv1_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_4)
    conv1_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), strides=1, activation='elu', use_bias=True, bias_initializer='he_normal')(conv1_e_5)

    # Batch normalization to get mean 0 activations
    batch1 = BatchNormalization()(conv1_e_6)

    # Second encoding block
    # Convolutions
    conv2_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv2_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_1)
    conv2_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_2)

    # Pooling
    pool2_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv2_e_3)

    # Convolutions
    conv2_e_4 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool2_e_1)
    conv2_e_5 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_4)
    conv2_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_5)

    batch2 = BatchNormalization()(conv2_e_6)

    # Concatenate
    path_combined = Concatenate(axis=4)([batch1, batch2])

    # Combined convolutions
    conv_7 = Conv3D(filters=32, kernel_initializer='he_normal', kernel_size=(3, 3, 3), activation='elu',
                    strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(path_combined)
    conv_8 = Conv3D(filters=64, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_7)
    conv_9 = Conv3D(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_8)

    batch3 = BatchNormalization()(conv_9)

    # Bidirectional-lstm block
    bi_dir_1 = Bidirectional(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', kernel_initializer='glorot_normal',
                                        recurrent_activation='hard_sigmoid',recurrent_initializer='he_normal', padding='same', use_bias=True, bias_initializer='glorot_normal', return_sequences=True))(batch3)

    # Deconvolution
    d_conv = Conv3DTranspose(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, use_bias=True, bias_initializer='he_normal')(bi_dir_1)
    d_conv_1 = Conv3DTranspose(filters=64, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv)
    d_conv_2 = Conv3DTranspose(filters=32, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_1)

    # first decoding block
    dconv1_e_1 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_2)
    dconv1_e_2 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_1)
    dconv1_e_3 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_2)

    upsample_e_1 = UpSampling3D(size=(2, 2, 2))(dconv1_e_3)

    dconv1_e_4 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(upsample_e_1)
    dconv1_e_5 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_4)

    batchnorm = BatchNormalization()(dconv1_e_5)

    mask = Conv3DTranspose(filters=4, kernel_initializer='glorot_normal', kernel_size=(
        3, 3, 3), activation='softmax', strides=1, padding='valid', use_bias=True, bias_initializer='glorot_normal')(batchnorm)

    model = Model(inputs=input_layer, outputs=mask)
    return model


In [47]:
#Bidirectional 2 path lstm cascaded

def cascaded_bilstm():
    input_layer = Input((128, 128, 128, 3))
    # first encoding block
    # Convolution layers
    conv1_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv1_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_1)
    conv1_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_2)
    conv1_e_4 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_3)
    conv1_e_5 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_4)

    # Downsample
    pool1_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv1_e_5)

    # Convolution Layers
    conv1_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool1_e_1)
    conv1_e_7 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_6)
    conv1_e_8 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), strides=1, activation='elu', use_bias=True, bias_initializer='he_normal')(conv1_e_7)
    conv1_e_9 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_8)
    conv1_e_10 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv1_e_9)

    # Batch normalization to get mean 0 activations
    batch1 = BatchNormalization()(conv1_e_10)

    # Second encoding block
    # Convolutions
    conv2_e_1 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(input_layer)
    conv2_e_2 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_1)
    conv2_e_3 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_2)
    conv2_e_4 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_3)
    conv2_e_5 = Conv3D(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_4)

    # Pooling
    pool2_e_1 = MaxPooling3D(pool_size=(2, 2, 2), strides=2)(conv2_e_5)

    # Convolutions
    conv2_e_6 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(pool2_e_1)
    conv2_e_7 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_6)
    conv2_e_8 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_7)
    conv2_e_9 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_8)
    conv2_e_10 = Conv3D(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv2_e_9)

    batch2 = BatchNormalization()(conv2_e_10)

    # Concatenate
    path_combined = Concatenate(axis=4)([batch1, batch2])

    # Combined convolutions
    conv_7 = Conv3D(filters=32, kernel_initializer='he_normal', kernel_size=(3, 3, 3), activation='elu',
                    strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(path_combined)
    conv_8 = Conv3D(filters=64, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_7)
    conv_9 = Conv3D(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(conv_8)

    batch3 = BatchNormalization()(conv_9)

    # Bidirectional-lstm block
    bi_dir_1 = Bidirectional(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', kernel_initializer='glorot_normal',
                                        recurrent_activation='hard_sigmoid', recurrent_initializer='he_normal', padding='same', use_bias=True, bias_initializer='glorot_normal', return_sequences=True))(batch3)
    bi_dir_2 = Bidirectional(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', kernel_initializer='glorot_normal',
                                        recurrent_activation='hard_sigmoid', recurrent_initializer='he_normal', padding='same', use_bias=True, bias_initializer='glorot_normal', return_sequences=True))(batch3)

    # Combined layer(attention)
    bidir_comb = Add()([bi_dir_1, bi_dir_2])

    # Deconvolution
    d_conv = Conv3DTranspose(filters=128, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, use_bias=True, bias_initializer='he_normal')(bidir_comb)
    d_conv_1 = Conv3DTranspose(filters=64, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv)
    d_conv_2 = Conv3DTranspose(filters=32, kernel_initializer='he_normal', activation='elu',
                               strides=1, kernel_size=(3, 3, 3), padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_1)

    # first decoding block
    dconv1_e_1 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(d_conv_2)
    dconv1_e_2 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_1)
    dconv1_e_3 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_2)
    dconv1_e_4 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_3)
    dconv1_e_5 = Conv3DTranspose(filters=16, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_4)

    upsample_e_1 = UpSampling3D(size=(2, 2, 2))(dconv1_e_5)

    dconv1_e_6 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(upsample_e_1)
    dconv1_e_7 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_6)
    dconv1_e_8 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_7)
    dconv1_e_9 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_8)
    dconv1_e_10 = Conv3DTranspose(filters=8, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='elu', strides=1, padding='valid', use_bias=True, bias_initializer='he_normal')(dconv1_e_9)

    batchnorm = BatchNormalization()(dconv1_e_10)

    # Feature learning feature mapping
    fl1 = Conv3D(filters=4, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='relu', padding='same', strides=1, use_bias=True, bias_initializer='he_normal')(batchnorm)
    fl1_batch = BatchNormalization()(fl1)

    fl2 = Conv3D(filters=4, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='relu', padding='same', strides=1, use_bias=True, bias_initializer='he_normal')(fl1_batch)
    fl2_batch = BatchNormalization()(fl2)

    fl3 = Conv3D(filters=4, kernel_initializer='he_normal', kernel_size=(
        3, 3, 3), activation='relu', padding='same', strides=1, use_bias=True, bias_initializer='he_normal')(fl2_batch)
    fl3_batch = BatchNormalization()(fl3)

    mask = Conv3D(filters=4, kernel_initializer='glorot_normal', kernel_size=(
        3, 3, 3), activation='softmax', strides=1, padding='same', use_bias=True, bias_initializer='glorot_normal')(fl3_batch)

    model = Model(inputs=input_layer, outputs=mask)
    return model


In [48]:

def ResidualBlockEnc(input_layer, no_of_filters):

    conv_1 = Conv3D(filters=no_of_filters, kernel_size=(3, 3, 3), padding="same", activation="relu",
                    use_bias=True, bias_initializer="he_normal", kernel_initializer="he_normal")(input_layer)
    BN_1 = BatchNormalization()(conv_1)
    conv_2 = Conv3D(filters=no_of_filters, kernel_size=(3, 3, 3), padding="same", activation="relu",
                    use_bias=True, bias_initializer="he_normal", kernel_initializer="he_normal")(BN_1)
    BN_2 = BatchNormalization()(conv_2)
    conv_skip = Conv3D(filters=no_of_filters, kernel_size=(
        1, 1, 1), bias_initializer="he_normal", kernel_initializer="he_normal")(input_layer)

    add_layer = Add()([BN_2, conv_skip])
    act = Activation(tf.keras.activations.relu)(add_layer)
    act=BatchNormalization()(act)

    return act


def ResidualBlockDec(input_layer, no_of_filters):

    deconv_1 = Conv3DTranspose(filters=no_of_filters, kernel_size=(3, 3, 3), padding="same", activation="relu",
                               use_bias=True, bias_initializer="he_normal", kernel_initializer="he_normal")(input_layer)
    BN_1 = BatchNormalization()(deconv_1)
    deconv_2 = Conv3DTranspose(filters=no_of_filters, kernel_size=(3, 3, 3), padding="same", activation="relu",
                               use_bias=True, bias_initializer="he_normal", kernel_initializer="he_normal")(BN_1)
    BN_2 = BatchNormalization()(deconv_2)
    deconv_skip = Conv3DTranspose(filters=no_of_filters, kernel_size=(
        1, 1, 1), bias_initializer="he_normal", kernel_initializer="he_normal")(input_layer)

    add_layer = Add()([BN_2, deconv_skip])
    act = Activation(tf.keras.activations.relu)(add_layer)
    act=BatchNormalization()(act)

    return act


def encoder(input_layer):

    res_block_1 = ResidualBlockEnc(input_layer, no_of_filters=8)
    res_block_2 = ResidualBlockEnc(res_block_1, no_of_filters=8)

    down_1 = MaxPooling3D(pool_size=(2, 2, 2))(res_block_2)

    res_block_3 = ResidualBlockEnc(down_1, no_of_filters=16)
    res_block_4 = ResidualBlockEnc(res_block_3, no_of_filters=16)

    down_2 = MaxPooling3D(pool_size=(2, 2, 2))(res_block_4)

    res_block_5 = ResidualBlockEnc(down_2, no_of_filters=32)
    res_block_6 = ResidualBlockEnc(res_block_5, no_of_filters=32)
    
    down_3=MaxPooling3D(pool_size=(2,2,2))(res_block_6)

    return down_3


def decoder(input_layer):
    up_1 = UpSampling3D(size=(2, 2, 2))(input_layer)

    res_block_1 = ResidualBlockDec(up_1, no_of_filters=32)
    res_block_2 = ResidualBlockDec(res_block_1, no_of_filters=32)

    up_2 = UpSampling3D(size=(2, 2, 2))(res_block_2)

    res_block_3 = ResidualBlockDec(up_2, no_of_filters=16)
    res_block_4 = ResidualBlockDec(res_block_3, no_of_filters=16)

    up_3 = UpSampling3D(size=(2, 2, 2))(res_block_4)

    res_block_5 = ResidualBlockDec(up_3, no_of_filters=8)
    res_block_6 = ResidualBlockDec(res_block_5, no_of_filters=8)

    return res_block_6


def lstms(input_layer):
    bi_dir_1 = Bidirectional(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', kernel_initializer='glorot_normal',
                                        recurrent_activation='hard_sigmoid', recurrent_initializer='glorot_normal', padding='same', use_bias=True, bias_initializer='glorot_normal', return_sequences=True))(input_layer)

    return bi_dir_1


def convolution_blocks(input_layer):
    conv = Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu',
                  kernel_initializer='he_normal', use_bias=True, bias_initializer='he_normal')(input_layer)

    batchnorm = BatchNormalization()(conv)

    return batchnorm


def make_model():
    input_layer = Input((128, 128, 128, 3))
    input_layer=BatchNormalization()(input_layer)
    # Encoding
    enc1 = encoder(input_layer)
    enc2 = encoder(input_layer)
    enc = Concatenate(axis=4)([enc1, enc2])

    # Bilstm layer
    bilstm = lstms(enc)

    # Decoder
    dec1 = decoder(bilstm)
   
    combined_layer = Conv3DTranspose(filters=4, kernel_size=(3, 3, 3), padding="same", activation="relu",
                                     use_bias=True, bias_initializer="he_normal", kernel_initializer="he_normal")(dec1)
    combined_layer=BatchNormalization()(combined_layer)

    # Few convolution la
    conv1 = convolution_blocks(combined_layer)
    conv2 = convolution_blocks(conv1)
    conv3 = convolution_blocks(conv2)
        
    
    output_layer = Conv3D(filters=4, kernel_size=(1, 1, 1), kernel_initializer='glorot_normal',
                          padding='same', activation='softmax', use_bias=True, bias_initializer='glorot_normal')(conv3)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model


In [49]:
#Compute steps per epoch
def steps(m,batchsize):
    return (m+batchsize-1)//batchsize

In [50]:
#Main program
sample_paths=sorted(glob("../input/mri-data/Data/Train/samples_processed/*.npy"))
masks_paths=sorted(glob("../input/mri-data/Data/Train/masks_reformatted/*.npy"))


In [51]:
#Train-test split
train_sample_paths,test_sample_paths=train_test_split(sample_paths,test_size=0.20)

In [52]:
len(train_sample_paths)

295

In [53]:
traingen=batch_generator(train_sample_paths,4)
testgen=batch_generator(test_sample_paths,4)

In [54]:
sample,masks=next(traingen)

In [22]:
sample.shape,masks.shape

((4, 128, 128, 128, 3), (4, 128, 128, 128, 4))

In [19]:
model=make_model()


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  656         ['input_2[0][0]']                
                                8, 8)                                                             
                                                                                                  
 conv3d_18 (Conv3D)             (None, 128, 128, 12  656         ['input_2[0][0]']                
                                8, 8)                                                         

In [55]:
train_steps=steps(len(train_sample_paths),4)
test_steps=steps(len(test_sample_paths),4)

In [56]:
import datetime

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss=[total_gen_dice_loss],metrics=[total_gen_dice_coef,tf.keras.metrics.CategoricalAccuracy()])

#callbacks
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
schedule=tf.keras.callbacks.LearningRateScheduler(scheduler)
plateau_handler=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=3,factor=0.1)
nanterminate=tf.keras.callbacks.TerminateOnNaN()
earlystop=tf.keras.callbacks.EarlyStopping(monitor='val_loss')
checkpoints=tf.keras.callbacks.ModelCheckpoint(filepath="/kaggle/working/models/2pathcnn_bilstm",save_weights_only=False,save_freq='epoch',save_best_only=True,monitor='total_gen_dice_coef',mode='max',verbose=1)

In [57]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [23]:
tf.test.is_gpu_available()

True

In [59]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))


In [58]:
session.close()

In [113]:
history=model.fit(traingen,steps_per_epoch=train_steps,epochs=40,validation_data=testgen,validation_steps=test_steps,verbose=1,callbacks=[tensorboard_callback,plateau_handler,schedule,checkpoints,nanterminate])

Epoch 1/40


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_31/1720159941.py", line 48, in total_gen_dice_loss  *
        weights=get_class_weight(ytrue)
    File "/tmp/ipykernel_31/3202338059.py", line 15, in get_class_weight  *
        if k==1:

    ValueError: condition of if statement expected to be `tf.bool` scalar, got Tensor("total_gen_dice_loss/while/while/Equal:0", shape=(128, 128, 4), dtype=bool); to check for None, use `is not None`


In [60]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title("Loss curve for train")
plt.xlabel("Epochs")
plt.ylabel("Loss")


NameError: name 'history' is not defined

In [ ]:
model=tf.keras.models.load_model("/kaggle/working/models/2pathcnn_bilstm",custom_objects={"dice_coef":dice_coef,"overall_loss":overall_loss})

In [ ]:
predgen=batch_generator(test_sample_paths,1)
k=model.evaluate(predgen,verbose=1)

In [ ]:
samp,mask=next(predgen)

In [ ]:
samp.shape,mask.shape

In [ ]:
k=model.predict(samp)

In [ ]:
k

In [ ]:
pred=np.reshape(k,(128*128*128,4))

In [ ]:
pred
    

In [ ]:
for i in pred:
    t=max(i)
    for j in range(4):
        if(i[j]==t):
            i[j]=1
        else:
            i[j]=0

In [ ]:
pred_r=np.reshape(pred,(128,128,128,4))

In [ ]:
pred_r

In [ ]:
mask

dice_coef(m

In [ ]:
k=dice_coef(mask[0],pred_r)

In [ ]:
k

In [ ]:
plt.imshow(mask[0,:,:,60,0])

In [ ]:
plt.imshow(pred_r[:,:,60,0])

In [ ]:
plt.imshow(samp[0,:,:,50,2],cmap='gray')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss curve for train")
plt.xlabel("Epochs")
plt.ylabel("Loss")

In [ ]:
plt.plot(history.history['val_dice_coef'])
plt.plot(history.history['dice_coef'])
plt.xlabel("Epochs")
plt.ylabel("Dice score")

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

In [ ]:
plt.plot(history.history['overall_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Weighted categorical and dice loss")